# Engineering Formulas
Mechanical / piping engineering calculations.

In [1]:
import numpy as np

## Basics

---
### Velocity / Diameter / Flowrate

Simply formula manipulation to make it easy to switch between velocity / diameter and flowrate.

#### Formula

$$ Q = AV  $$

#### Parameters
- A : float
    - Area of pipe, [m²]
- V : float
    - Velocity, [m/s]

#### Returns
- Q : float
    - Flowrate, [m³/s]

#### Python Formula

In [13]:
def find_dia(Q_m3h, V_ms):
    area = Q_m3h / 60 / 60 / V_ms
    dia_mm = np.sqrt((4*area)/(np.pi)) * 1000
    return print('Diameter =',round(dia_mm,2),'mm')

def find_flow(dia_mm, V_ms):
    area = (np.pi*(dia_mm/1000)**2)/4
    Q_m3h = V_ms * area * 60 * 60
    return print('Flowrate =',round(Q_m3h,2),'m³/s')

def find_vel(dia_mm, Q_m3h):
    area = (np.pi*(dia_mm/1000)**2)/4
    V_ms = Q_m3h / 60 / 60 / area
    return print('Velocity =',round(V_ms,2),'m/s')

#### Example

In [18]:
find_vel(dia_mm = 20, Q_m3h = 300)

Velocity = 265.26 m/s


## Fluid Mechanics

---
### Friction loss through pipe

#### Formula

$$ h_L = f\frac{L}{D}\frac{V^2}{2g}  $$

#### Parameters
- f : float
    - Darcy Friction Factor, [-]
- L : float
    - Length of pipe, [m]
- D : float
    - Inner diameter of pipe, [m]
- V : float
    - Velocity of fluid, [m/s]
- g : float
    - Gravitational acceleration = 9.81, [m/s²]

#### Returns
- h_l : float
    - Head loss, [m]

#### Python Formula

In [ ]:
def friction_loss(f, L, D, V, g = 9.81):

    h_l = f * (L/D) * ((V**2) / (2*g))

    return h_l

#### Example

In [ ]:
friction_loss(0.019, 1000, 0.254, 2.3)

20.168637177233574

#### References
[1] Yunus Cengel, Fluid Mechanics. eq. (8-24). 4th edition. 2017.

---
### Flow Discharge Factor - K Factor

#### Formula

$$ K_L = \frac{h_L}{V^2/(2g)} $$

#### Parameters
- h_L : float
    - Head losses through fitting, [m]
- V : float
    - Velocity of fluid, [m/s]
- g : float
    - Gravitational acceleration = 9.81, [m/s²]

#### Returns
- K_L : float
    - Flow Discharge Factor, [-]

#### Python Formula

In [20]:
def k_factor(h_L, V, g = 9.81):
  K_L = (h_L*2*g)/(V**2)
  return K_L

#### Example

In [ ]:
k_factor(0.5, 3)

1.09

#### References
[1] Yunus Cengel, Fluid Mechanics. eq. (8–55). 4th edition. 2017.

---
### Bernoulli's principle
Bernoulli's principle states that an increase in the speed of a fluid occurs simultaneously with a decrease in static pressure or a decrease in the fluid's potential energy.

#### Formula

$$ P_1 + \frac{1}{2} \rho v_1^2 +\rho g h = 
  P_2 + \frac{1}{2} \rho v_2^2 +\rho g h $$

#### Parameters
- P : float
   - Pressure Energy, [Pa]
- $\rho$ : float
   - Density of the fluid, [kg/m³]
- v : float
   - Velocity of fluid, [m/s]
- h : float
  - Physical Height, [m]
- g : float
    - Gravitational acceleration = 9.81, [m/s²]

In [19]:
def pumphead (delta_h, fric_f, L_m, d_mm , Ktotal, V, P1, P2, rho):
        return ((P2-P1)*1000/(rho*g)) + (delta_h) + (( fric_f * (L/(d/1000)) + Ktotal ) * ((V**2)/(2*g)))

#### References
[1] Yunus Cengel, Fluid Mechanics. eq. (8–55). 4th edition. 2017.